In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Cristóvão Beirão da Cruz e Silva <cbeiraod@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

#############################################################################
# Modified for ETROC2 I2C testing in jupyter notebooks, Murtaza Safdari
#############################################################################

## Imports

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.cm as cm

import logging
import numpy as np
import time
import datetime
from tqdm import tqdm
import os
os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import multiprocessing
import run_script
import importlib
importlib.reload(run_script)
import pandas
from pathlib import Path
import subprocess
import sqlite3
from fnmatch import fnmatch

import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
from i2c_gui.chips.etroc2_chip import register_decoding

## Specify board name

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_name = "ET2-W42-IP7-2"
TID_str = "HV210V"
chip_figname = f"LowBiasCurrent_{TID_str}_{chip_name}"
chip_figtitle= "LowBiasCurrent "+TID_str+chip_name

fig_outdir = Path('../ETROC-figures')
fig_outdir = fig_outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

## Set defaults

In [ ]:
# 'If set, the full log will be saved to a file (i.e. the log level is ignored)'
log_file = False
# 'Set the logging level. Default: WARNING',
#  ["CRITICAL","ERROR","WARNING","INFO","DEBUG","TRACE","DETAILED_TRACE","NOTSET"]
log_level_text = "WARNING"
# 'The port name the USB-ISS module is connected to. Default: COM3'
port = "/dev/ttyACM0"
# I2C addresses for the pixel block and WS
chip_address = 0x62
# chip_address = 0x60
ws_address = None

In [ ]:
if log_file:
    logging.basicConfig(filename='logging.log', filemode='w', encoding='utf-8', level=logging.NOTSET)
    log_level = 0
else:
    log_level = 0
    if log_level_text == "CRITICAL":
        log_level=50
    elif log_level_text == "ERROR":
        log_level=40
    elif log_level_text == "WARNING":
        log_level=30
    elif log_level_text == "INFO":
        log_level=20
    elif log_level_text == "DEBUG":
        log_level=10
    elif log_level_text == "TRACE":
        log_level=8
    elif log_level_text == "DETAILED_TRACE":
        log_level=5
    elif log_level_text == "NOTSET":
        log_level=0
    logging.basicConfig(format='%(asctime)s - %(levelname)s:%(name)s:%(message)s')

i2c_gui.__no_connect__ = False  # Set to fake connecting to an ETROC2 device
i2c_gui.__no_connect_type__ = "echo"  # for actually testing readback
#i2c_gui.__no_connect_type__ = "check"  # default behaviour

## Start logger and connect

In [ ]:
logger = logging.getLogger("Script_Logger")

Script_Helper = i2c_gui.ScriptHelper(logger)

## USB ISS connection
conn = i2c_gui.Connection_Controller(Script_Helper)
conn.connection_type = "USB-ISS"
conn.handle: USB_ISS_Helper
conn.handle.port = port
conn.handle.clk = 100

conn.connect()

In [ ]:
chip = i2c_gui.chips.ETROC2_Chip(parent=Script_Helper, i2c_controller=conn)
chip.config_i2c_address(chip_address)  # Not needed if you do not access ETROC registers (i.e. only access WS registers)
# chip.config_waveform_sampler_i2c_address(ws_address)  # Not needed if you do not access WS registers

logger.setLevel(log_level)

## Useful Functions

In [ ]:
def pixel_decoded_register_write(decodedRegisterName, data_to_write):
    bit_depth = register_decoding["ETROC2"]["Register Blocks"]["Pixel Config"][decodedRegisterName]["bits"]
    handle = chip.get_decoded_indexed_var("ETROC2", "Pixel Config", decodedRegisterName)
    chip.read_decoded_value("ETROC2", "Pixel Config", decodedRegisterName)
    if len(data_to_write)!=bit_depth: print("Binary data_to_write is of incorrect length for",decodedRegisterName, "with bit depth", bit_depth)
    data_hex_modified = hex(int(data_to_write, base=2))
    if(bit_depth>1): handle.set(data_hex_modified)
    elif(bit_depth==1): handle.set(data_to_write)
    else: print(decodedRegisterName, "!!!ERROR!!! Bit depth <1, how did we get here...")
    chip.write_decoded_value("ETROC2", "Pixel Config", decodedRegisterName)

def pixel_decoded_register_read(decodedRegisterName, key, need_int=False):
    handle = chip.get_decoded_indexed_var("ETROC2", f"Pixel {key}", decodedRegisterName)
    chip.read_decoded_value("ETROC2", f"Pixel {key}", decodedRegisterName)
    if(need_int): return int(handle.get(), base=16)
    else: return handle.get()

def peripheral_decoded_register_write(decodedRegisterName, data_to_write):
    bit_depth = register_decoding["ETROC2"]["Register Blocks"]["Peripheral Config"][decodedRegisterName]["bits"]
    handle = chip.get_decoded_display_var("ETROC2", "Peripheral Config", decodedRegisterName)
    chip.read_decoded_value("ETROC2", "Peripheral Config", decodedRegisterName)
    if len(data_to_write)!=bit_depth: print("Binary data_to_write is of incorrect length for",decodedRegisterName, "with bit depth", bit_depth)
    data_hex_modified = hex(int(data_to_write, base=2))
    if(bit_depth>1): handle.set(data_hex_modified)
    elif(bit_depth==1): handle.set(data_to_write)
    else: print(decodedRegisterName, "!!!ERROR!!! Bit depth <1, how did we get here...")
    chip.write_decoded_value("ETROC2", "Peripheral Config", decodedRegisterName)

def peripheral_decoded_register_read(decodedRegisterName, key, need_int=False):
    handle = chip.get_decoded_display_var("ETROC2", f"Peripheral {key}", decodedRegisterName)
    chip.read_decoded_value("ETROC2", f"Peripheral {key}", decodedRegisterName)
    if(need_int): return int(handle.get(), base=16)
    else: return handle.get()

# Pixel ID Check

In [ ]:
Failure_map = np.zeros((16,16))
row_indexer_handle,_,_ = chip.get_indexer("row")  # Returns 3 parameters: handle, min, max
column_indexer_handle,_,_ = chip.get_indexer("column")
for row in range(16):
    for col in range(16):
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)
        fetched_row = pixel_decoded_register_read("PixelID-Row", "Status", need_int=True)
        fetched_col = pixel_decoded_register_read("PixelID-Col", "Status", need_int=True)
        if(row!=fetched_row or col!=fetched_col):
            print("Fail!", row, col, fetched_row, fetched_col)
            Failure_map[15-row,15-col] = 1

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(dpi=75, figsize=(8,8))
gs = fig.add_gridspec(1,1)

ax0 = fig.add_subplot(gs[0,0])
ax0.set_title("Pixel ID Failure Map")
img0 = ax0.imshow(Failure_map, interpolation='none')
ax0.set_aspect("equal")
ax0.get_yaxis().set_visible(False)
ax0.get_xaxis().set_visible(False)
divider = make_axes_locatable(ax0)
cax = divider.append_axes('right', size="5%", pad=0.05)
fig.colorbar(img0, cax=cax, orientation="vertical")
plt.show()

## Set Peripheral Registers

In [ ]:
peripheral_decoded_register_write("EFuse_Prog", format(0x00017f0f, '032b'))
peripheral_decoded_register_write("singlePort", '1')
peripheral_decoded_register_write("serRateLeft", '00')
peripheral_decoded_register_write("serRateRight", '00')
peripheral_decoded_register_write("onChipL1AConf", '00')
peripheral_decoded_register_write("PLL_ENABLEPLL", '1')
peripheral_decoded_register_write("chargeInjectionDelay", format(0x0a, '05b'))
peripheral_decoded_register_write("triggerGranularity", format(0x01, '03b')) # only for trigger bit

## Automatic threshold calibration

In [ ]:
BL_map_THCal = np.zeros((16,16))
NW_map_THCal = np.zeros((16,16))

In [ ]:
row_indexer_handle,_,_ = chip.get_indexer("row")  # Returns 3 parameters: handle, min, max
column_indexer_handle,_,_ = chip.get_indexer("column")
data = []
# Loop for threshold calibration
for row in tqdm(range(16), desc=" row", position=0):
    for col in tqdm(range(16), desc=" col", position=1, leave=False):
# for index,row,col in zip(tqdm(range(16)), row_list, col_list):
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)
        # Maybe required to make this work
        # pixel_decoded_register_write("enable_TDC", "0")
        # pixel_decoded_register_write("testMode_TDC", "0")
        # Enable THCal clock and buffer, disable bypass
        pixel_decoded_register_write("CLKEn_THCal", "1")
        pixel_decoded_register_write("BufEn_THCal", "1")
        pixel_decoded_register_write("Bypass_THCal", "0")
        pixel_decoded_register_write("TH_offset", format(0x04, '06b'))
        # Reset the calibration block (active low)
        pixel_decoded_register_write("RSTn_THCal", "0")
        pixel_decoded_register_write("RSTn_THCal", "1")
        # Start and Stop the calibration, (25ns x 2**15 ~ 800 us, ACCumulator max is 2**15)
        pixel_decoded_register_write("ScanStart_THCal", "1")
        pixel_decoded_register_write("ScanStart_THCal", "0")
        # Check the calibration done correctly
        if(pixel_decoded_register_read("ScanDone", "Status")!="1"): print("!!!ERROR!!! Scan not done!!!")
        BL_map_THCal[row, col] = pixel_decoded_register_read("BL", "Status", need_int=True)
        NW_map_THCal[row, col] = pixel_decoded_register_read("NW", "Status", need_int=True)
        data += [{
            'col': col,
            'row': row,
            'baseline': BL_map_THCal[row, col],
            'noise_width': NW_map_THCal[row, col],
            'timestamp': datetime.datetime.now(),
            'chip_name': chip_name,
        }]
        # Disable clock and buffer before charge injection 
        pixel_decoded_register_write("CLKEn_THCal", "0") 
        pixel_decoded_register_write("BufEn_THCal", "0")
        
        pixel_decoded_register_write("Bypass_THCal", "1")
        pixel_decoded_register_write("DAC", format(0x3ff, '010b'))
        # Set Charge Inj Q to 15 fC
        pixel_decoded_register_write("QSel", format(0x0e, '05b'))

        time.sleep(0.1)

BL_df = pandas.DataFrame(data = data)

In [ ]:
fig = plt.figure(dpi=200, figsize=(10,10))
gs = fig.add_gridspec(1,2)

ax0 = fig.add_subplot(gs[0,0])
# ax0.set_title("BL (DAC LSB), "+chip_figtitle, size=8)
img0 = ax0.imshow(BL_map_THCal, interpolation='none')
ax0.set_aspect("equal")
ax0.invert_xaxis()
ax0.invert_yaxis()
plt.xticks(range(16), range(16), rotation="vertical")
plt.yticks(range(16), range(16))
divider = make_axes_locatable(ax0)
cax = divider.append_axes('right', size="5%", pad=0.05)
fig.colorbar(img0, cax=cax, orientation="vertical")

ax1 = fig.add_subplot(gs[0,1])
# ax1.set_title("NW (DAC LSB), "+chip_figtitle, size=8)
img1 = ax1.imshow(NW_map_THCal, interpolation='none')
ax1.set_aspect("equal")
ax1.invert_xaxis()
ax1.invert_yaxis()
plt.xticks(range(16), range(16), rotation="vertical")
plt.yticks(range(16), range(16))
divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size="5%", pad=0.05)
fig.colorbar(img1, cax=cax, orientation="vertical")

for x in range(16):
    for y in range(16):
        # if(BL_map_THCal.T[x,y]==0): continue
        ax0.text(x,y,f"{BL_map_THCal.T[x,y]:.0f}", c="white", size=5, rotation=45, fontweight="bold", ha="center", va="center")
        ax1.text(x,y,f"{NW_map_THCal.T[x,y]:.0f}", c="white", size=5, rotation=45, fontweight="bold", ha="center", va="center")
plt.savefig(fig_path+"/BL_NW_"+chip_figname+"_"+datetime.date.today().isoformat()+".png")
# plt.show()

### Store BL, NW dataframe for later use

In [ ]:
outdir = Path('../ETROC-Data')
outdir = outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
outdir.mkdir(exist_ok=True)
outfile = outdir / (chip_name+TID_str+"_BaselineAt_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M") + ".csv")
BL_df.to_csv(outfile, index=False)

### Store BL, NW dataframe in database

In [ ]:
note = 'PreTID'+chip_name
new_columns = {
    'note': f'{note}',
}

for col in new_columns:
    BL_df[col] = new_columns[col]

outdir = Path('../ETROC-History')
outfile = outdir / 'BaselineHistory.sqlite'

init_cmd = [
    'cd ' + str(outdir.resolve()),
    'git stash -u',
    'git pull',
]
end_cmd = [
    'cd ' + str(outdir.resolve()),
    'git add BaselineHistory.sqlite',
    'git commit -m "Added new history entry"',
    'git push',
    'git stash pop',
    'git stash clear',
]
init_cmd = [x + '\n' for x in init_cmd]
end_cmd  = [x + '\n' for x in end_cmd]

p = subprocess.Popen(
    '/bin/bash',
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    encoding="utf-8",
    )

for cmd in init_cmd:
    p.stdin.write(cmd + "\n")
p.stdin.close()
p.wait()

print(p.stdout.read())

with sqlite3.connect(outfile) as sqlconn:
    BL_df.to_sql('baselines', sqlconn, if_exists='append', index=False)

p = subprocess.Popen(
    '/bin/bash',
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    encoding="utf-8",
    )

for cmd in end_cmd:
    p.stdin.write(cmd + "\n")
p.stdin.close()
p.wait()

p.stdin.close()

print(p.stdout.read())


## Disable all pixels

In [ ]:
# row_indexer_handle,_,_ = chip.get_indexer("row")
# column_indexer_handle,_,_ = chip.get_indexer("column")
# column_indexer_handle.set(0)
# row_indexer_handle.set(0)

# broadcast_handle,_,_ = chip.get_indexer("broadcast")
# broadcast_handle.set(True)
# pixel_decoded_register_write("disDataReadout", "1")
# broadcast_handle.set(True)
# pixel_decoded_register_write("QInjEn", "0")
# broadcast_handle.set(True)
# pixel_decoded_register_write("disTrigPath", "1")
# # Release the maximum and minimum range for trigger and data
# pixel_decoded_register_write("upperTOATrig", format(0x3ff, '010b'))
# broadcast_handle.set(True)
# pixel_decoded_register_write("lowerTOATrig", format(0x000, '010b'))
# broadcast_handle.set(True)
# pixel_decoded_register_write("upperTOTTrig", format(0x1ff, '09b'))
# broadcast_handle.set(True)
# pixel_decoded_register_write("lowerTOTTrig", format(0x000, '09b'))
# broadcast_handle.set(True)
# pixel_decoded_register_write("upperCalTrig", format(0x3ff, '010b'))
# broadcast_handle.set(True)
# pixel_decoded_register_write("lowerCalTrig", format(0x000, '010b'))
# broadcast_handle.set(True)
# pixel_decoded_register_write("upperTOA", format(0x3ff, '010b'))
# broadcast_handle.set(True)
# pixel_decoded_register_write("lowerTOA", format(0x000, '010b'))
# broadcast_handle.set(True)
# pixel_decoded_register_write("upperTOT", format(0x1ff, '09b'))
# broadcast_handle.set(True)
# pixel_decoded_register_write("lowerTOT", format(0x000, '09b'))
# broadcast_handle.set(True)
# pixel_decoded_register_write("upperCal", format(0x3ff, '010b'))
# broadcast_handle.set(True)
# pixel_decoded_register_write("lowerCal", format(0x000, '010b'))

In [ ]:
row_indexer_handle,_,_ = chip.get_indexer("row")  # Returns 3 parameters: handle, min, max
column_indexer_handle,_,_ = chip.get_indexer("column")
# Loop for setting upper/lower TDC bounds
for row in tqdm(range(16), desc=" row", position=0):
    for col in tqdm(range(16), desc=" col", position=1, leave=False):
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)
        pixel_decoded_register_write("disDataReadout", "1")
        pixel_decoded_register_write("QInjEn", "0")
        pixel_decoded_register_write("disTrigPath", "1")
        pixel_decoded_register_write("upperTOATrig", format(0x3ff, '010b'))
        pixel_decoded_register_write("lowerTOATrig", format(0x000, '010b'))
        pixel_decoded_register_write("upperTOTTrig", format(0x1ff, '09b'))
        pixel_decoded_register_write("lowerTOTTrig", format(0x000, '09b'))
        pixel_decoded_register_write("upperCalTrig", format(0x3ff, '010b'))
        pixel_decoded_register_write("lowerCalTrig", format(0x000, '010b'))
        pixel_decoded_register_write("upperTOA", format(0x3ff, '010b'))
        pixel_decoded_register_write("lowerTOA", format(0x000, '010b'))
        pixel_decoded_register_write("upperTOT", format(0x1ff, '09b'))
        pixel_decoded_register_write("lowerTOT", format(0x000, '09b'))
        pixel_decoded_register_write("upperCal", format(0x3ff, '010b'))
        pixel_decoded_register_write("lowerCal", format(0x000, '010b'))

# One time run to set fpga firmware

In [ ]:
parser = run_script.getOptionParser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname 192.168.2.7 -t 15 -o CanBeRemoved -v -w --compressed_translation -s 0x000C -p 0x000f -d 0x0800 --clear_fifo".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_Start_LEDs'))
process.start()

IPC_queue.put('start L1A trigger bit')
while not IPC_queue.empty():
    pass
time.sleep(5)
IPC_queue.put('stop DAQ')
IPC_queue.put('stop L1A trigger bit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

In [ ]:
column_indexer_handle.set(9)
row_indexer_handle.set(14)
print("Pixel:",col,row)
# Enable charge injection
pixel_decoded_register_write("disDataReadout", "0")
pixel_decoded_register_write("QInjEn", "1")
pixel_decoded_register_write("disTrigPath", "0")

# one time run for set fpga firmware
parser = run_script.getOptionParser()
(options, args) = parser.parse_args(args=f"--useIPC --hostname 192.168.2.7 -t 15 -o CanBeRemoved -v -w --compressed_translation -s 0x000C -p 0x000f -d 0x0800 --clear_fifo".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_Start_LEDs'))
process.start()

IPC_queue.put('start L1A trigger bit')
while not IPC_queue.empty():
    pass
time.sleep(10)
IPC_queue.put('stop DAQ')
IPC_queue.put('stop L1A trigger bit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

pixel_decoded_register_write("disDataReadout", "1")
pixel_decoded_register_write("QInjEn", "0")
pixel_decoded_register_write("disTrigPath", "1")

# Run DAQ scanning by row to study multiple pixels TOT and TOA

### Define DAQ function

In [ ]:
def run_daq(timePerPixel, deadTime, triggerBitDelay, dirname):
    
    time_per_pixel = timePerPixel
    dead_time_per_pixel = deadTime
    trigger_bit_delay = triggerBitDelay
    total_scan_time = time_per_pixel + dead_time_per_pixel
    outname = dirname

    today = datetime.date.today()
    todaystr = "../ETROC-Data/" + today.isoformat() + "_Array_Test_Results/"
    base_dir = Path(todaystr)
    base_dir.mkdir(exist_ok=True)

    parser = run_script.getOptionParser() 
    # (options, args) = parser.parse_args(args=f"--useIPC --hostname 192.168.2.7 -t {int(total_scan_time)} -o {outname} -v -w -s 0x000C -p 0x000f --compressed_translation -d 0x0800 --clear_fifo".split())
    (options, args) = parser.parse_args(args=f"--useIPC --hostname 192.168.2.7 -t {int(total_scan_time)} -o {outname} -v -w -s 0x000C -p 0x000f -d 0x0800 --clear_fifo".split())
    IPC_queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process'))
    process.start()

    IPC_queue.put('start L1A trigger bit')
    while not IPC_queue.empty():
        pass

    # delay = '000011'+format(trigger_bit_delay, '010b')
    # hex_delay = hex(int(delay, base=2))
    # IPC_queue.put(f'change delay {hex_delay}') 
    # while not IPC_queue.empty():
    #     pass

    time.sleep(time_per_pixel)
    IPC_queue.put('stop L1A trigger bit')

    time.sleep(1)
    IPC_queue.put('stop DAQ')
    while not IPC_queue.empty():
        pass

    IPC_queue.put('allow threads to exit')

    process.join()

### Actual DAQ run

In [ ]:
# Actual DAQ run
for i in range(1):

    # Disable pixels for clean start
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")
    column_indexer_handle.set(0)
    row_indexer_handle.set(0)

    broadcast_handle,_,_ = chip.get_indexer("broadcast")
    broadcast_handle.set(True)
    pixel_decoded_register_write("disDataReadout", "1")
    broadcast_handle.set(True)
    pixel_decoded_register_write("QInjEn", "0")
    broadcast_handle.set(True)
    pixel_decoded_register_write("disTrigPath", "1")

    scan_list = list(zip(np.full(16, i), np.arange(16)))
    print(scan_list)

    for row, col in scan_list:
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)

        print(f"Enabling Pixel ({row},{col})")

        pixel_decoded_register_write("Bypass_THCal", "0")               # Bypass threshold calibration -> manual DAC setting
        pixel_decoded_register_write("QSel", format(0x0e, '05b'))       # Ensure we inject 0 fC of charge
        pixel_decoded_register_write("TH_offset", format(0x0c, '06b'))  # Offset used to add to the auto BL for real triggering
        pixel_decoded_register_write("disDataReadout", "0")             # ENable readout
        pixel_decoded_register_write("QInjEn", "1")                     # ENable charge injection for the selected pixel
        pixel_decoded_register_write("L1Adelay", format(0x01f5, '09b')) # Change L1A delay - circular buffer in ETROC2 pixel
        pixel_decoded_register_write("disTrigPath", "0")                # Enable trigger path

    run_daq(10, 6, 485, f'TID_testing_candidate_{chip_name.replace("_","")}_'+TID_str+'_R{str(i)}_CX')

# Qinj S Curve Scan

## Define Pixel for ACC and DAC scan

In [ ]:
# DAC_row_list = [15, 15, 15, 15, 14, 14, 14, 14]
# DAC_col_list = [6, 7, 8, 9, 6, 7, 8, 9]
DAC_row_list = [14, 15]
DAC_col_list = [6, 6]
# DAC_row_list = [0]
# DAC_col_list = [0]
DAC_scan_list = list(zip(DAC_col_list, DAC_row_list))
print(DAC_scan_list)

## Simple Scan To Measure The Noise

In [ ]:
thresholds = np.arange(-15,15,1) # relative to BL
# thresholds = np.arange(0,1,1) # BL only
scan_name = "E2_testing_VRef_SCurve_Noise_"+TID_str
fpga_time = 3
QInj = 0

row_indexer_handle,_,_ = chip.get_indexer("row")  # Returns 3 parameters: handle, min, max
column_indexer_handle,_,_ = chip.get_indexer("column")

today = datetime.date.today()
todaystr = "../ETROC-Data/" + today.isoformat() + "_Array_Test_Results/"
base_dir = Path(todaystr)
base_dir.mkdir(exist_ok=True)

data = []

# Loop for enable/disable charge injection per pixel (single!!!)
for index, row, col in zip(range(len(DAC_row_list)), DAC_row_list, DAC_col_list):  
    print("Pixel:",col,row)
    column_indexer_handle.set(col)
    row_indexer_handle.set(row)
    # Ensure charge injection is disabled
    pixel_decoded_register_write("disDataReadout", "0")
    pixel_decoded_register_write("QInjEn", "0")
    pixel_decoded_register_write("disTrigPath", "0")

    # Bypass Cal Threshold
    pixel_decoded_register_write("Bypass_THCal", "1")

    # start FPGA state
    threshold_name = scan_name+f'_Pixel_C{col}_R{row}_Noise_HVoff_pf_hits'
    (options, args) = parser.parse_args(args=f"--useIPC --hostname 192.168.2.7 -o {threshold_name} -v -w --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit 3 --fpga_data --check_trigger_link_at_end --nodaq --clear_fifo".split())
    IPC_queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_link'))
    process.start()
    process.join()
    
    for DAC in tqdm(thresholds[:], desc=f'DAC Loop for Pixel {col},{row}', leave=False):
        DAC = int(DAC+BL_map_THCal[row][col])
        print("DAC", DAC)

        # Set the DAC v, Qinj {Qinj}fCalue to the value being scanned
        pixel_decoded_register_write("DAC", format(DAC, '010b'))

        (options, args) = parser.parse_args(args=f"--useIPC --hostname 192.168.2.7 -o {threshold_name} -v --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit {int(fpga_time)} --fpga_data --check_trigger_link_at_end --nodaq --DAC_Val {int(DAC)}".split())
        IPC_queue = multiprocessing.Queue()
        process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_Noise_{QInj}_{DAC}'))
        process.start()
        process.join()
        
    # Disable charge injection
    pixel_decoded_register_write("QInjEn", "0")
    pixel_decoded_register_write("disDataReadout", "1")
    pixel_decoded_register_write("disTrigPath", "1")

sCurve_df = pandas.DataFrame(data=data)

outdir = Path('../ETROC-Data')
outdir = outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
outdir.mkdir(exist_ok=True)
outfile = outdir / (scan_name + "_at_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M") + ".csv")
sCurve_df.to_csv(outfile, index=False)

### Choose Pixel To Plot Noise To Check Output

In [ ]:
row = 14
col = 8
DAC_plot_row_list = [row]
DAC_plot_col_list = [col]

In [ ]:
root = '../ETROC-Data'
file_pattern = "*FPGA_Data.dat"
BL = int(BL_map_THCal[row][col])
triggerbit_full_Scurve = {row:{col:{thr+BL:0 for thr in thresholds} for col in range(16)} for row in range(16)}
for index, row, col in zip((range(len(DAC_plot_row_list))), DAC_plot_row_list, DAC_plot_col_list):
    print(f'Pixel {col},{row}')
    path_pattern = f"*{today.isoformat()}_Array_Test_Results/{scan_name}_Pixel_C{col}_R{row}_Noise_HVoff_pf_hits"
    file_list = []
    for path, subdirs, files in os.walk(root):
        if not fnmatch(path, path_pattern): continue
        for name in files:
            pass
            if fnmatch(name, file_pattern):
                file_list.append(os.path.join(path, name))
                print(file_list[-1])
    total_files = len(file_list)
    for file_index, file_name in enumerate(file_list):
        print(f"{file_index+1}/{total_files}")
        with open(file_name) as infile:
            for line in infile:
                text_list = line.split(',')
                FPGA_triggerbit = int(text_list[5])
                DAC = int(text_list[-1])
                if DAC == -1: continue
                try:
                    triggerbit_full_Scurve[row][col][DAC] += FPGA_triggerbit
                except:
                    print(f"Couldn't find DAC: {DAC} in file list")

In [ ]:
fig = plt.figure(dpi=200, figsize=(8,4.5))
gs = fig.add_gridspec(1,1)
u_cl = np.sort(np.unique(DAC_plot_col_list))
u_rl = np.sort(np.unique(DAC_plot_row_list))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        ax0.axvline(BL_map_THCal[row][col], color='k', label="THCal BL", lw=0.7)
        ax0.axvline(BL_map_THCal[row][col]+NW_map_THCal[row][col], color='r', ls='-', label="THCal NW", lw=0.7)
        ax0.axvline(BL_map_THCal[row][col]-NW_map_THCal[row][col], color='r', ls='-', lw=0.7)
        ax0.plot([thr+BL for thr in thresholds], triggerbit_full_Scurve[row][col].values(), '.-', color='b',lw=0.5,markersize=2)
        ax0.set_xlabel("DAC Value [decimal]")
        ax0.set_ylabel("Data Counts [decimal]")
        # ax0.text(0.7, 0.8, f"Pixel {row},{col}", transform=ax0.transAxes)
        plt.legend(loc="upper right")
        plt.yscale("log")
    plt.title(f"{chip_figtitle}, Pixel ({row},{col}) Noise Only S-Curve")
    plt.tight_layout()
# plt.savefig(fig_path+"/Noise_S-Curve_"+chip_figname+"_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")

## DAC scan using counter

### Define DACs for scanning near BL

In [ ]:
# relative to BL
min_threshold = -20
BL_scan_max = 20
BL_scan_step = 2

BL_thresholds = {
    1: np.arange(min_threshold,  min_threshold + BL_scan_max, BL_scan_step),
    5: np.arange(min_threshold,  min_threshold + BL_scan_max, BL_scan_step),
    6: np.arange(min_threshold,  min_threshold + BL_scan_max, BL_scan_step),
    8: np.arange(min_threshold,  min_threshold + BL_scan_max, BL_scan_step),
    10: np.arange(min_threshold, min_threshold + BL_scan_max, BL_scan_step),
    12: np.arange(min_threshold, min_threshold + BL_scan_max, BL_scan_step),
    15: np.arange(min_threshold, min_threshold + BL_scan_max, BL_scan_step),
    17: np.arange(min_threshold, min_threshold + BL_scan_max, BL_scan_step),
    20: np.arange(min_threshold, min_threshold + BL_scan_max, BL_scan_step),
    22: np.arange(min_threshold, min_threshold + BL_scan_max, BL_scan_step),
    25: np.arange(min_threshold, min_threshold + BL_scan_max, BL_scan_step),
    27: np.arange(min_threshold, min_threshold + BL_scan_max, BL_scan_step),
    30: np.arange(min_threshold, min_threshold + BL_scan_max, BL_scan_step),
}


### Define DACs for scanning Pedestal

In [ ]:
# relative to BL, sensor with HV
min_threshold = min_threshold + BL_scan_max
pedestal_scan_step = 2

pedestal_thresholds = {
    1: np.arange(min_threshold,  min_threshold + 40, pedestal_scan_step),
    5: np.arange(min_threshold,  min_threshold + 40, pedestal_scan_step),
    6: np.arange(min_threshold,  min_threshold + 40, pedestal_scan_step),
    8: np.arange(min_threshold,  min_threshold + 40, pedestal_scan_step),
    10: np.arange(min_threshold, min_threshold + 70, pedestal_scan_step),
    12: np.arange(min_threshold, min_threshold + 90, pedestal_scan_step),
    15: np.arange(min_threshold, min_threshold + 60, pedestal_scan_step),
    17: np.arange(min_threshold, min_threshold + 130, pedestal_scan_step),
    20: np.arange(min_threshold, min_threshold + 140, pedestal_scan_step),
    22: np.arange(min_threshold, min_threshold + 80, pedestal_scan_step),
    25: np.arange(min_threshold, min_threshold + 200, pedestal_scan_step),
    27: np.arange(min_threshold, min_threshold + 100, pedestal_scan_step),
    30: np.arange(min_threshold, min_threshold + 280, pedestal_scan_step),
}

In [ ]:
# relative to BL, bare chip
min_threshold = min_threshold + BL_scan_max
pedestal_scan_step = 2

pedestal_thresholds = {
    1: np.arange(min_threshold,  min_threshold + 60, pedestal_scan_step),
    5: np.arange(min_threshold,  min_threshold + 60, pedestal_scan_step),
    6: np.arange(min_threshold,  min_threshold + 60, pedestal_scan_step),
    8: np.arange(min_threshold,  min_threshold + 60, pedestal_scan_step),
    10: np.arange(min_threshold, min_threshold + 70, pedestal_scan_step),
    12: np.arange(min_threshold, min_threshold + 90, pedestal_scan_step),
    15: np.arange(min_threshold, min_threshold + 120, pedestal_scan_step),
    17: np.arange(min_threshold, min_threshold + 130, pedestal_scan_step),
    20: np.arange(min_threshold, min_threshold + 140, pedestal_scan_step),
    22: np.arange(min_threshold, min_threshold + 160, pedestal_scan_step),
    25: np.arange(min_threshold, min_threshold + 200, pedestal_scan_step),
    27: np.arange(min_threshold, min_threshold + 210, pedestal_scan_step),
    30: np.arange(min_threshold, min_threshold + 280, pedestal_scan_step),
}

### Combine Thresholds

In [ ]:
thresholds = {}
for QInj in BL_thresholds:
    thresholds[QInj] = list(BL_thresholds[QInj])
    for DAC in pedestal_thresholds[QInj]:
        thresholds[QInj].append(DAC)

### Define Charges

In [ ]:
# Full Charges
# QInjEns = [5, 6, 8, 10, 12, 15, 17, 20, 22, 25, 27, 30]
# Recommend for TID
# QInjEns = [8, 10, 15, 22, 27]
# Single
QInjEns = [22]
num_thr = 0
for QInj in QInjEns:
    num_thr += len(thresholds[QInj])
num_pix = len(DAC_scan_list)
print(f"Will scan {num_thr} thresholds and {num_pix} pixels")
print(f"Expected scan time is {num_thr*num_pix*5./60.} minutes")

## Run QInj+DAC Scan

#### scan THoffset

In [ ]:
#define offsets
offset_scan_step = 1
offset_max = 40
offsets = {
    1: np.arange(0,  offset_max, offset_scan_step),
    5: np.arange(0,  offset_max, offset_scan_step),
    6: np.arange(0,  offset_max, offset_scan_step),
    8: np.arange(0,  offset_max, offset_scan_step),
    10: np.arange(0, offset_max, offset_scan_step),
    12: np.arange(0, offset_max, offset_scan_step),
    15: np.arange(0, offset_max, offset_scan_step),
    17: np.arange(0, offset_max, offset_scan_step),
    20: np.arange(0, offset_max, offset_scan_step),
    22: np.arange(0, offset_max, offset_scan_step),
    25: np.arange(0, offset_max, offset_scan_step),
    27: np.arange(0, offset_max, offset_scan_step),
    30: np.arange(0, offset_max, offset_scan_step),
}

In [ ]:
scan_name = "E2_testing_VRef_SCurve_offset_"+TID_str
fpga_time = 3

row_indexer_handle,_,_ = chip.get_indexer("row")  # Returns 3 parameters: handle, min, max
column_indexer_handle,_,_ = chip.get_indexer("column")

today = datetime.date.today()
todaystr = "../ETROC-Data/" + today.isoformat() + "_Array_Test_Results/"
base_dir = Path(todaystr)
base_dir.mkdir(exist_ok=True)

data = []

# Loop for enable/disable charge injection per pixel (single!!!)
for index, row, col in zip(range(len(DAC_row_list)), DAC_row_list[:], DAC_col_list[:]): 
    column_indexer_handle.set(col)
    row_indexer_handle.set(row)
    print("Pixel:",col,row)
    # Enable charge injection
    pixel_decoded_register_write("disDataReadout", "0")
    pixel_decoded_register_write("QInjEn", "1")
    pixel_decoded_register_write("disTrigPath", "0")
    # Bypass Cal Threshold
    pixel_decoded_register_write("Bypass_THCal", "0")
    # for QInj in tqdm(QInjEns, desc=f'Charge Loop for Pixel {col},{row}', leave=False):
    for QInj in QInjEns[:]:
        print(f"Charge: {QInj} fC")
        # Modifying charge injected
        pixel_decoded_register_write("QSel", format(QInj, '05b'))
        threshold_name = scan_name+f'_Pixel_C{col}_R{row}_QInj_{QInj}_HVoff_pf_hits'
        (options, args) = parser.parse_args(args=f"--useIPC --hostname 192.168.2.7 -o {threshold_name} -v -w --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit 3 --fpga_data_QInj --check_trigger_link_at_end --nodaq --clear_fifo".split())
        IPC_queue = multiprocessing.Queue()
        process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_link'))
        process.start()
        process.join()
        
        for offset in tqdm(offsets[QInj][:], desc=f'Offset Loop for Pixel ({col},{row}) & Charge {QInj} fC', leave=False):
            pixel_decoded_register_write("TH_offset", format(offset, '06b'))
            TH = pixel_decoded_register_read("TH","Status",True)
            (options, args) = parser.parse_args(args=f"--useIPC --hostname 192.168.2.7 -o {threshold_name} -v --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit {int(fpga_time)} --fpga_data_QInj --check_trigger_link_at_end --nodaq --DAC_Val {int(TH)}".split())
            IPC_queue = multiprocessing.Queue()
            process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_{QInj}_{TH}'))
            process.start()
            process.join()

            # pixel_decoded_register_write("upperTOATrig", format(0x3ff, '010b'))
            # pixel_decoded_register_write("lowerTOATrig", format(0x000, '010b'))
            # pixel_decoded_register_write("upperTOTTrig", format(0x1ff, '09b'))
            # pixel_decoded_register_write("lowerTOTTrig", format(0x000, '09b'))
            # pixel_decoded_register_write("upperCalTrig", format(0x3ff, '010b'))
            # pixel_decoded_register_write("lowerCalTrig", format(0x000, '010b'))

            print("TH_offset",pixel_decoded_register_read("TH_offset","Config",True),"BL",pixel_decoded_register_read("BL","Status",True),"TH",TH,"\n"
            "upperTOATrig",pixel_decoded_register_read("upperTOATrig", "Config", True),
            "lowerTOATrig",pixel_decoded_register_read("lowerTOATrig", "Config", True),
            "upperTOTTrig",pixel_decoded_register_read("upperTOTTrig", "Config", True),
            "lowerTOTTrig",pixel_decoded_register_read("lowerTOTTrig", "Config", True),
            "upperCalTrig",pixel_decoded_register_read("upperCalTrig", "Config", True),
            "lowerCalTrig",pixel_decoded_register_read("lowerCalTrig", "Config", True))
            
    # Disable charge injection
    pixel_decoded_register_write("QInjEn", "0")
    pixel_decoded_register_write("disDataReadout", "1")
    pixel_decoded_register_write("disTrigPath", "1")

sCurve_df = pandas.DataFrame(data=data)

outdir = Path('../ETROC-Data')
outdir = outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
outdir.mkdir(exist_ok=True)
outfile = outdir / (scan_name + "_at_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M") + ".csv")
sCurve_df.to_csv(outfile, index=False)

In [ ]:
# DAC_plot_row_list = [15, 15, 15, 15, 14, 14, 14, 14]
# DAC_plot_col_list = [6, 7, 8, 9, 6, 7, 8, 9]
DAC_plot_row_list = [14, 15]
DAC_plot_col_list = [6, 6]
# row = 15
# col = 6
# DAC_plot_row_list = [row]
# DAC_plot_col_list = [col]

In [ ]:
root = '../ETROC-Data'
file_pattern = "*FPGA_Data.dat"
hitmap_full_Scurve = {row:{col:{q:{thr+int(BL_map_THCal[row][col]):0 for thr in offsets[q]} for q in QInjEns} for col in range(16)} for row in range(16)}
triggerbit_full_Scurve = {row:{col:{q:{thr+int(BL_map_THCal[row][col]):0 for thr in offsets[q]} for q in QInjEns} for col in range(16)} for row in range(16)}
data_full_Scurve = {row:{col:{q:{thr+int(BL_map_THCal[row][col]):0 for thr in offsets[q]} for q in QInjEns} for col in range(16)} for row in range(16)}
for index, row, col in zip((range(len(DAC_plot_row_list))), DAC_plot_row_list, DAC_plot_col_list):
    BL = int(BL_map_THCal[row][col])
    for QInj in (QInjEns):
        print(f'Pixel {col},{row} - {QInj} fC')
        path_pattern = f"*{today.isoformat()}_Array_Test_Results/E2_testing_VRef_SCurve_offset_{TID_str}_Pixel_C{col}_R{row}_QInj_{QInj}_HVoff_pf_hits"
        file_list = []
        for path, subdirs, files in os.walk(root):
            if not fnmatch(path, path_pattern): continue
            for name in files:
                pass
                if fnmatch(name, file_pattern):
                    file_list.append(os.path.join(path, name))
                    # print(file_list[-1])
        total_files = len(file_list)
        for file_index, file_name in enumerate(file_list):
            # print(f"{file_index+1}/{total_files}")
            with open(file_name) as infile:
                for line in infile:
                    text_list = line.split(',')
                    FPGA_state = text_list[0]
                    FPGA_data = int(text_list[3])
                    FPGA_triggerbit = int(text_list[5])
                    DAC = int(text_list[-1])
                    if DAC == -1: continue
                    try:
                        triggerbit_full_Scurve[row][col][QInj][DAC] += FPGA_triggerbit
                        data_full_Scurve[row][col][QInj][DAC] += FPGA_data
                        hitmap_full_Scurve[row][col][QInj][DAC] += 1
                    except:
                        print(f"Couldn't find DAC: {DAC} in file list")

In [ ]:
colors = [plt.cm.viridis(i) for i in np.linspace(0,1,len(QInjEns))]
# fig = plt.figure(dpi=200, figsize=(8,4.5))
# gs = fig.add_gridspec(1,1)
# u_cl = np.sort(np.unique(DAC_plot_col_list))
# u_rl = np.sort(np.unique(DAC_plot_row_list))

fig = plt.figure(dpi=200, figsize=(10*len(DAC_plot_col_list),10*len(DAC_plot_row_list)))
gs = fig.add_gridspec(len(DAC_plot_row_list),len(DAC_plot_col_list))
u_cl = np.sort(np.unique(DAC_plot_col_list))
u_rl = np.sort(np.unique(DAC_plot_row_list))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        ax0.axvline(BL_map_THCal[row][col], color='k', label="THCal BL", lw=0.7)
        ax0.axvline(BL_map_THCal[row][col]+NW_map_THCal[row][col], color='r', ls='-', label="THCal NW", lw=0.7)
        ax0.axvline(BL_map_THCal[row][col]-NW_map_THCal[row][col], color='r', ls='-', lw=0.7)
        for i, QInj in enumerate(QInjEns[:]):
            ax0.plot([thr+BL for thr in offsets[QInj]], data_full_Scurve[row][col][QInj].values(), 'd-', color=colors[i], label=f"{QInj} fC Data",lw=0.5,markersize=1.5)
            ax0.plot([thr+BL for thr in offsets[QInj]], triggerbit_full_Scurve[row][col][QInj].values(), '.--', color=colors[i], label=f"{QInj} fC TrigB",lw=0.5,markersize=2)
        ax0.set_xlabel("DAC Value [decimal]")
        ax0.set_ylabel("Data Counts [decimal]")
        # ax0.text(0.7, 0.8, f"Pixel {row},{col}", transform=ax0.transAxes)
        ax0.set_ylim(bottom=1)
        plt.legend(loc="upper right")
        plt.yscale("log")
    plt.title(f"{chip_figtitle}, Pixel ({row},{col}) Full S-Curve")
    plt.tight_layout()
plt.savefig(fig_path+"/Full_S-Curve_"+chip_figname+"_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")

#### scan DAC

In [ ]:
scan_name = "E2_testing_VRef_SCurve_"+TID_str
fpga_time = 3

row_indexer_handle,_,_ = chip.get_indexer("row")  # Returns 3 parameters: handle, min, max
column_indexer_handle,_,_ = chip.get_indexer("column")

today = datetime.date.today()
todaystr = "../ETROC-Data/" + today.isoformat() + "_Array_Test_Results/"
base_dir = Path(todaystr)
base_dir.mkdir(exist_ok=True)

data = []

# Loop for enable/disable charge injection per pixel (single!!!)
# for index, row, col in zip(tqdm(range(len(DAC_row_list)), desc=f'Pixel Loop', leave=True), DAC_row_list, DAC_col_list):
for index, row, col in zip(range(len(DAC_row_list)), DAC_row_list, DAC_col_list): 
    column_indexer_handle.set(col)
    row_indexer_handle.set(row)
    print("Pixel:",col,row)
    # Enable charge injection
    pixel_decoded_register_write("disDataReadout", "0")
    pixel_decoded_register_write("QInjEn", "1")
    pixel_decoded_register_write("disTrigPath", "0")
    # Bypass Cal Threshold
    pixel_decoded_register_write("Bypass_THCal", "1")
    # for QInj in tqdm(QInjEns, desc=f'Charge Loop for Pixel {col},{row}', leave=False):
    for QInj in QInjEns:
        print(f"Charge: {QInj} fC")
        # Modifying charge injected
        pixel_decoded_register_write("QSel", format(QInj, '05b'))
        threshold_name = scan_name+f'_Pixel_C{col}_R{row}_QInj_{QInj}_HVoff_pf_hits'
        (options, args) = parser.parse_args(args=f"--useIPC --hostname 192.168.2.7 -o {threshold_name} -v -w --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit 3 --fpga_data_QInj --check_trigger_link_at_end --nodaq --clear_fifo".split())
        IPC_queue = multiprocessing.Queue()
        process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_link'))
        process.start()
        process.join()
        
        for DAC in tqdm(thresholds[QInj][:], desc=f'DAC Loop for Pixel ({col},{row}) & Charge {QInj} fC', leave=False):
            DAC = int(BL_map_THCal[row][col]+DAC)
            # print(QInj, DAC)
            # Set the DAC v, Qinj {Qinj}fCalue to the value being scanned
            pixel_decoded_register_write("DAC", format(DAC, '010b'))
            # Adding this to allow the new DAC setting to propogate to the counters
            # time.sleep(0.5)
            (options, args) = parser.parse_args(args=f"--useIPC --hostname 192.168.2.7 -o {threshold_name} -v --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit {int(fpga_time)} --fpga_data_QInj --check_trigger_link_at_end --nodaq --DAC_Val {int(DAC)}".split())
            IPC_queue = multiprocessing.Queue()
            process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_{QInj}_{DAC}'))
            process.start()
            process.join()
            
    # Disable charge injection
    pixel_decoded_register_write("QInjEn", "0")
    pixel_decoded_register_write("disDataReadout", "1")
    pixel_decoded_register_write("disTrigPath", "1")

sCurve_df = pandas.DataFrame(data=data)

outdir = Path('../ETROC-Data')
outdir = outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
outdir.mkdir(exist_ok=True)
outfile = outdir / (scan_name + "_at_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M") + ".csv")
sCurve_df.to_csv(outfile, index=False)

### Choose Pixel To Plot Full Scan Output

In [ ]:
# DAC_plot_row_list = [15, 15, 15, 15, 14, 14, 14, 14]
# DAC_plot_col_list = [6, 7, 8, 9, 6, 7, 8, 9]
DAC_plot_row_list = [14, 15]
DAC_plot_col_list = [8, 8]
# row = 15
# col = 6
# DAC_plot_row_list = [row]
# DAC_plot_col_list = [col]

In [ ]:
root = '../ETROC-Data'
file_pattern = "*FPGA_Data.dat"
hitmap_full_Scurve = {row:{col:{q:{thr+BL:0 for thr in thresholds[q]} for q in QInjEns} for col in range(16)} for row in range(16)}
triggerbit_full_Scurve = {row:{col:{q:{thr+BL:0 for thr in thresholds[q]} for q in QInjEns} for col in range(16)} for row in range(16)}
data_full_Scurve = {row:{col:{q:{thr+BL:0 for thr in thresholds[q]} for q in QInjEns} for col in range(16)} for row in range(16)}
for index, row, col in zip((range(len(DAC_plot_row_list))), DAC_plot_row_list, DAC_plot_col_list):
    BL = int(BL_map_THCal[row][col])
    for QInj in (QInjEns):
        print(f'Pixel {col},{row} - {QInj} fC')
        path_pattern = f"*{today.isoformat()}_Array_Test_Results/E2_testing_VRef_SCurve_{TID_str}_Pixel_C{col}_R{row}_QInj_{QInj}_HVoff_pf_hits"
        file_list = []
        for path, subdirs, files in os.walk(root):
            if not fnmatch(path, path_pattern): continue
            for name in files:
                pass
                if fnmatch(name, file_pattern):
                    file_list.append(os.path.join(path, name))
                    # print(file_list[-1])
        total_files = len(file_list)
        for file_index, file_name in enumerate(file_list):
            # print(f"{file_index+1}/{total_files}")
            with open(file_name) as infile:
                for line in infile:
                    text_list = line.split(',')
                    FPGA_state = text_list[0]
                    FPGA_data = int(text_list[3])
                    FPGA_triggerbit = int(text_list[5])
                    DAC = int(text_list[-1])
                    if DAC == -1: continue
                    try:
                        triggerbit_full_Scurve[row][col][QInj][DAC] += FPGA_triggerbit
                        data_full_Scurve[row][col][QInj][DAC] += FPGA_data
                        hitmap_full_Scurve[row][col][QInj][DAC] += 1
                    except:
                        print(f"Couldn't find DAC: {DAC} in file list")

In [ ]:
colors = [plt.cm.viridis(i) for i in np.linspace(0,1,len(QInjEns))]
# fig = plt.figure(dpi=200, figsize=(8,4.5))
# gs = fig.add_gridspec(1,1)
# u_cl = np.sort(np.unique(DAC_plot_col_list))
# u_rl = np.sort(np.unique(DAC_plot_row_list))

fig = plt.figure(dpi=200, figsize=(10*len(DAC_plot_col_list),10*len(DAC_plot_row_list)))
gs = fig.add_gridspec(len(DAC_plot_row_list),len(DAC_plot_col_list))
u_cl = np.sort(np.unique(DAC_plot_col_list))
u_rl = np.sort(np.unique(DAC_plot_row_list))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        ax0.axvline(BL_map_THCal[row][col], color='k', label="THCal BL", lw=0.7)
        ax0.axvline(BL_map_THCal[row][col]+NW_map_THCal[row][col], color='r', ls='-', label="THCal NW", lw=0.7)
        ax0.axvline(BL_map_THCal[row][col]-NW_map_THCal[row][col], color='r', ls='-', lw=0.7)
        for i, QInj in enumerate(QInjEns[0:]):
            ax0.plot([thr+BL for thr in thresholds[QInj]], data_full_Scurve[row][col][QInj].values(), 'd-', color=colors[i], label=f"{QInj} fC Data",lw=0.5,markersize=1.5)
            ax0.plot([thr+BL for thr in thresholds[QInj]], triggerbit_full_Scurve[row][col][QInj].values(), '.--', color=colors[i], label=f"{QInj} fC TrigB",lw=0.5,markersize=2)
        ax0.set_xlabel("DAC Value [decimal]")
        ax0.set_ylabel("Data Counts [decimal]")
        # ax0.text(0.7, 0.8, f"Pixel {row},{col}", transform=ax0.transAxes)
        plt.legend(loc="upper right")
        plt.yscale("log")
    plt.title(f"{chip_figtitle}, Pixel ({row},{col}) Full S-Curve")
    plt.tight_layout()
plt.savefig(fig_path+"/Full_S-Curve_"+chip_figname+"_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")

## DAC Binary Search for Qinj Turn On

In [ ]:
DAC_scan_max = 600

In [ ]:
# Full Charges
# QInjEns = [3, 5, 6, 8, 10, 12, 15, 17, 20, 22, 25, 27, 30]
# Recommend for TID
QInjEns = [5, 6, 8, 15, 22, 27]
# Single
# QInjEns = [20]

### Binary search for trigger bit fall off

In [ ]:
# scan_name = "E2_testing_VRef_SCurve_BinarySearch_"+TID_str
# fpga_time = 3

# row_indexer_handle,_,_ = chip.get_indexer("row")  # Returns 3 parameters: handle, min, max
# column_indexer_handle,_,_ = chip.get_indexer("column")

# today = datetime.date.today()
# todaystr = "../ETROC-Data/" + today.isoformat() + "_Array_Test_Results/"
# base_dir = Path(todaystr)
# base_dir.mkdir(exist_ok=True)

# data = []

# # Loop for enable/disable charge injection per pixel (single!!!)
# for index, row, col in zip(tqdm(range(len(DAC_row_list)), desc=f'Pixel Loop', leave=True), DAC_row_list, DAC_col_list):  
#     column_indexer_handle.set(col)
#     row_indexer_handle.set(row)
#     print("Pixel:",col,row)
#     # Enable charge injection
#     pixel_decoded_register_write("disDataReadout", "0")
#     pixel_decoded_register_write("QInjEn", "1")
#     pixel_decoded_register_write("disTrigPath", "0")
#     # Bypass Cal Threshold
#     pixel_decoded_register_write("Bypass_THCal", "1")
#     for QInj in tqdm(QInjEns, desc=f'Charge Loop for Pixel {col},{row}', leave=False):
#         # Modifying charge injected
#         pixel_decoded_register_write("QSel", format(QInj, '05b'))
#         threshold_name = scan_name+f'_Pixel_C{col}_R{row}_QInj_{QInj}_HVoff_pf_hits'
#         (options, args) = parser.parse_args(args=f"--useIPC --hostname 192.168.2.7 -o {threshold_name} -v -w --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit 3 --fpga_data_QInj --check_trigger_link_at_end --nodaq --clear_fifo".split())
#         IPC_queue = multiprocessing.Queue()
#         process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_link'))
#         process.start()
#         process.join()
        
#         a = BL_map_THCal[row][col]
#         b = DAC_scan_max
#         while b-a>1:
#             DAC = int(np.floor((a+b)/2))
#             print("QInj", QInj, "DAC", DAC, "a", a, "b", b)
#             # print("QInj", QInj, "DAC", DAC)
#             # Set the DAC v, Qinj {Qinj}fCalue to the value being scanned
#             pixel_decoded_register_write("DAC", format(DAC, '010b'))

#             (options, args) = parser.parse_args(args=f"--useIPC --hostname 192.168.2.7 -o {threshold_name} -v --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit {int(fpga_time)} --fpga_data_QInj --check_trigger_link_at_end --nodaq --DAC_Val {int(DAC)}".split())
#             IPC_queue = multiprocessing.Queue()
#             process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_{QInj}_{DAC}'))
#             process.start()
#             process.join()
            
#             continue_flag = False

#             root = '../ETROC-Data'
#             file_pattern = "*FPGA_Data.dat"
#             path_pattern = f"*{today.isoformat()}_Array_Test_Results/{threshold_name}"
#             file_list = []
#             for path, subdirs, files in os.walk(root):
#                 if not fnmatch(path, path_pattern): continue
#                 for name in files:
#                     pass
#                     if fnmatch(name, file_pattern):
#                         file_list.append(os.path.join(path, name))
#             total_files = len(file_list)
#             for file_index, file_name in enumerate(file_list):
#                 with open(file_name) as infile:
#                     last_line = infile.readlines()[-1]
#                     text_list = last_line.split(',')
#                     FPGA_state = text_list[0]
#                     line_DAC = int(text_list[6])
#                     if FPGA_state==0 or line_DAC!=DAC: 
#                         continue_flag=True
#                         continue
#                     FPGA_data = int(text_list[5])
#                     TDC_data = int(text_list[3])
#                     # Condition handling for Binary Search
#                     if(FPGA_data>0):
#                         a = DAC
#                     else:
#                         b = DAC                    
#             if(continue_flag): continue  
            
#     # Disable charge injection
#     pixel_decoded_register_write("QInjEn", "0")
#     pixel_decoded_register_write("disDataReadout", "1")
#     pixel_decoded_register_write("disTrigPath", "1")

# sCurve_df = pandas.DataFrame(data=data)

# outdir = Path('../ETROC-Data')
# outdir = outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
# outdir.mkdir(exist_ok=True)
# outfile = outdir / (scan_name + "_at_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M") + ".csv")
# sCurve_df.to_csv(outfile, index=False)

### Binary search for "TDC" data fall off

In [ ]:
scan_name = "E2_testing_VRef_SCurve_BinarySearch_"+TID_str
fpga_time = 3

row_indexer_handle,_,_ = chip.get_indexer("row")  # Returns 3 parameters: handle, min, max
column_indexer_handle,_,_ = chip.get_indexer("column")

today = datetime.date.today()
todaystr = "../ETROC-Data/" + today.isoformat() + "_Array_Test_Results/"
base_dir = Path(todaystr)
base_dir.mkdir(exist_ok=True)

data = []

# Loop for enable/disable charge injection per pixel (single!!!)
for index, row, col in zip(range(len(DAC_row_list)), DAC_row_list, DAC_col_list):  
    column_indexer_handle.set(col)
    row_indexer_handle.set(row)
    print("Pixel:",col,row)
    # Enable charge injection
    pixel_decoded_register_write("disDataReadout", "0")
    pixel_decoded_register_write("QInjEn", "1")
    pixel_decoded_register_write("disTrigPath", "0")
    # Bypass Cal Threshold
    pixel_decoded_register_write("Bypass_THCal", "1")
    for QInj in QInjEns:
        # Modifying charge injected
        pixel_decoded_register_write("QSel", format(QInj, '05b'))
        threshold_name = scan_name+f'_Pixel_C{col}_R{row}_QInj_{QInj}_HVoff_pf_hits'
        (options, args) = parser.parse_args(args=f"--useIPC --hostname 192.168.2.7 -o {threshold_name} -v -w --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit 3 --fpga_data_QInj --check_trigger_link_at_end --nodaq --clear_fifo".split())
        IPC_queue = multiprocessing.Queue()
        process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_link'))
        process.start()
        process.join()

        c = BL_map_THCal[row][col]
        d = DAC_scan_max
        header_max = -1
        while d-c>1:
            DAC = int(np.floor((c+d)/2))
            print("QInj", QInj, "DAC", DAC, "c", c, "d", d)
            # Set the DAC v, Qinj {Qinj}fCalue to the value being scanned
            pixel_decoded_register_write("DAC", format(DAC, '010b'))

            (options, args) = parser.parse_args(args=f"--useIPC --hostname 192.168.2.7 -o {threshold_name} -v --reset_till_trigger_linked -s 0x000C -p 0x000f -d 0x0800 -c 0x0001 --fpga_data_time_limit {int(fpga_time)} --fpga_data_QInj --check_trigger_link_at_end --nodaq --DAC_Val {int(DAC)}".split())
            IPC_queue = multiprocessing.Queue()
            process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_{QInj}_{DAC}'))
            process.start()
            process.join()
            
            continue_flag = False
            root = '../ETROC-Data'
            file_pattern = "*FPGA_Data.dat"
            path_pattern = f"*{today.isoformat()}_Array_Test_Results/{threshold_name}"
            file_list = []
            for path, subdirs, files in os.walk(root):
                if not fnmatch(path, path_pattern): continue
                for name in files:
                    pass
                    if fnmatch(name, file_pattern):
                        file_list.append(os.path.join(path, name))
            total_files = len(file_list)
            for file_index, file_name in enumerate(file_list):
                with open(file_name) as infile:
                    lines = infile.readlines()
                    last_line = lines[-1]
                    first_line = lines[0]
                    header_max = int(first_line.split(',')[4])
                    text_list = last_line.split(',')
                    FPGA_state = text_list[0]
                    line_DAC = int(text_list[-1])
                    if(FPGA_state==0 or line_DAC!=DAC): 
                        continue_flag=True
                        continue
                    TDC_data = int(text_list[3])
                    # header_data = int(text_list[4])
                    # Condition handling for Binary Search
                    if(TDC_data>=header_max/2.):
                        c = DAC
                    else:
                        d = DAC                    
            if(continue_flag): continue  
            
    # Disable charge injection
    pixel_decoded_register_write("QInjEn", "0")
    pixel_decoded_register_write("disDataReadout", "1")
    pixel_decoded_register_write("disTrigPath", "1")

sCurve_df = pandas.DataFrame(data=data)

outdir = Path('../ETROC-Data')
outdir = outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
outdir.mkdir(exist_ok=True)
outfile = outdir / (scan_name + "_at_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M") + ".csv")
sCurve_df.to_csv(outfile, index=False)

### Plot QInj vs DAC

In [ ]:
import scipy.stats as stats
QInj_DAC_map = {row:{col:{q:0 for q in QInjEns} for col in range(16)} for row in range(16)}
for index, row, col in zip(tqdm(range(len(DAC_row_list)), desc=f'Pixel Loop', leave=True), DAC_row_list, DAC_col_list):  
    print("Pixel:",col,row)
    for QInj in tqdm(QInjEns, desc=f'Charge Loop for Pixel {col},{row}', leave=False):
        threshold_name = scan_name+f'_Pixel_C{col}_R{row}_QInj_{QInj}_HVoff_pf_hits'
        path_pattern = f"*{today.isoformat()}_Array_Test_Results/{threshold_name}"
        file_list = []
        for path, subdirs, files in os.walk(root):
            if not fnmatch(path, path_pattern): continue
            for name in files:
                pass
                if fnmatch(name, file_pattern):
                    file_list.append(os.path.join(path, name))
        total_files = len(file_list)
        for file_index, file_name in enumerate(file_list):
            with open(file_name) as infile:
                last_line = infile.readlines()[-1]
                text_list = last_line.split(',')
                DAC = int(text_list[-1])
                QInj_DAC_map[row][col][QInj] = DAC
fig = plt.figure(dpi=200, figsize=(10*len(DAC_col_list),10*len(DAC_col_list)))
gs = fig.add_gridspec(len(DAC_row_list),len(DAC_col_list))
u_cl = np.sort(np.unique(DAC_col_list))
u_rl = np.sort(np.unique(DAC_row_list))
for ri,row in enumerate(u_rl):
    for ci,col in enumerate(u_cl):
        BL = int(np.floor(BL_map_THCal[row][col]))
        NW = abs(int(np.floor(NW_map_THCal[row][col])))
        ax0 = fig.add_subplot(gs[len(u_rl)-ri-1,len(u_cl)-ci-1])
        ax0.axhline(BL, color='k', lw=0.8, label=f"BL = {BL} DAC LSB")
        ax0.axhline(BL+NW, color='k',ls="--", lw=0.8, label=f"NW = {NW} DAC LSB")
        ax0.axhline(BL-NW, color='k',ls="--", lw=0.8)
        X = []
        Y = []
        for QInj in QInjEns:
            # tp_idx = turning_point(np.array(list(hit_counts[(col,row)][QInj].values())), thresholds[QInj])
            ax0.plot(QInj, QInj_DAC_map[row][col][QInj], 'ro')
            X.append(QInj)
            Y.append(QInj_DAC_map[row][col][QInj])
        X = np.array(X[:])
        Y = np.array(Y[:])
        (m, b), cov = np.polyfit(X, Y, 1, cov = True)
        n = Y.size
        Yfit = np.polyval((m,b), X)
        errorbars = np.sqrt(np.diag(cov))
        x_range = np.linspace(0, 35, 100)
        y_est = b + m*x_range
        resid = Y - Yfit
        s_err = np.sqrt(np.sum(resid**2)/(n - 2))
        t = stats.t.ppf(0.99, n - 2)
        ci = t * s_err * np.sqrt(    1/n + (X - np.mean(X))**2/np.sum((X-np.mean(X))**2)) # confidence interval
        pi = t * s_err * np.sqrt(1 + 1/n + (X - np.mean(X))**2/np.sum((X-np.mean(X))**2)) # prediction interval
        # cip= t * s_err * np.sqrt(    1/n + (X - np.mean(X))**2/(np.sum((X)**2)-n*np.sum((np.mean(X))**2)))
        ci2= t * s_err * np.sqrt(    1/n + (x_range - np.mean(X))**2/(np.sum((X)**2)-n*np.sum((np.mean(X))**2)))
        
        ax0.plot(x_range, y_est, 'b-', lw=-.8, label=f"DAC_TH = ({m:.3f}$\pm${errorbars[0]:.3f} [1/fC])$\cdot$Q + ({b:.3f}$\pm${errorbars[1]:.3f})")
        plt.fill_between(x_range, y_est+ci2, y_est-ci2, color='b',alpha=0.2, label="99% Confidence Interval on Linear Fit")
        ax0.set_xlabel("Charge Injected [fC]")
        ax0.set_ylabel("DAC Threshold [LSB]")
        plt.title(f"ET2 W36 IP5-18, Pixel ({row},{col}), Qinj S-Curve, PA Low Bias Current Mode")
        plt.legend(loc="upper left")
plt.tight_layout()
# plt.savefig("figures/q_vs_dac_scurve.pdf")
# plt.savefig(f"figures/q_vs_dac_scurve_({row},{col}).png")
plt.show()

# Disconnect chip

In [ ]:
conn.disconnect()